In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import sqlite3

C:\Users\37061\AppData\Local\Temp\ipykernel_1508\619177735.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
conn = sqlite3.connect('energy.db')

# Source: IEA World Energy Balances https://www.iea.org/data-and-statistics/data-product/world-energy-statistics-and-balances
# This data is released under a CC BY 4.0 licence: https://www.iea.org/terms/creative-commons-cc-licenses
df_energy_production = pd.read_sql_query('SELECT * FROM "Energy supply by source - Lithuania"', conn)

# Source: Our World in Data
df_renewables_by_source = pd.read_sql_query('SELECT * FROM "Renewables generated by source (OWD)"', conn)
df_electricity_by_source = pd.read_sql_query('SELECT * FROM "Electricity by soruce (OWD)"', conn)
df_share_electricity_renewables = pd.read_sql_query('SELECT * FROM "Share of electricity production from renewables (OWD)"', conn)
df_share_electricity_imports = pd.read_sql_query('SELECT * FROM "electricity-imports-share-demand (OWD)"', conn)

conn.close()

### Energy production by source in Lithuania

In [3]:
# Create traces for each energy source
traces = []
for column in df_energy_production.columns[1:-1]:
    trace = go.Bar(
        x=df_energy_production['Year'],
        y=df_energy_production[column],
        name=column
    )
    traces.append(trace)

layout = go.Layout(
    title='Energy Production by Source in Lithuania',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Energy (TJ)'),
    hovermode='closest',
    barmode='stack',  # Stack/group
    legend=dict(x=1.1, y=0.5),
    height=600
)

fig13 = go.Figure(data=traces, layout=layout)

# Reference data source
fig13.add_annotation(
    text="Data Source: IEA World Energy Balances 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)

fig13.update_layout(template="plotly_dark")
fig13.show()

### Electricity production by source in Lithuania

In [4]:
# Filter data for Lithuania
df_lithuania = df_electricity_by_source[df_electricity_by_source['Entity'] == 'Lithuania']

# Remove leading spaces from column names
df_lithuania.columns = df_lithuania.columns.str.strip()

# Selecting relevant columns for plotting
df_lithuania = df_lithuania[['Year', 'Coal (TWh)', 'Hydro (TWh)', 'Wind (TWh)', 'Solar (TWh)', 'Bioenergy (TWh)', 
                             'Nuclear (TWh)', 'Oil (TWh)', 'Gas (TWh)', 'Other renewables (TWh)']]

# Create traces for each source
traces = []
source_names = ['Coal', 'Hydro', 'Wind', 'Solar', 'Bioenergy', 'Nuclear', 'Oil', 'Gas', 'Other renewables']

for i, source in enumerate(source_names):
    trace = go.Bar(x=df_lithuania['Year'], y=df_lithuania[f'{source} (TWh)'], name=source)
    traces.append(trace)

# Create layout
layout = go.Layout(
    title='Electricity Production by Source in Lithuania',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Electricity Production'),
    barmode='stack',
    height=600
)

fig15 = go.Figure(data=traces, layout=layout)
fig15.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)
fig15.update_layout(template="plotly_dark")
fig15.show()

### Renewable energy production in Lithuania

In [5]:
# Define traces for each renewable energy source
filtered_data = df_renewables_by_source[df_renewables_by_source['Entity'].isin(['Lithuania'])]
trace_wind = go.Scatter(
    x=filtered_data['Year'],
    y=filtered_data['Electricity from wind - TWh'],
    mode='markers+lines',
    name='Wind',
    marker=dict(symbol='circle', size=8)
)

trace_solar = go.Scatter(
    x=filtered_data['Year'],
    y=filtered_data['Electricity from solar - TWh'],
    mode='markers+lines',
    name='Solar',
    marker=dict(symbol='circle', size=8)
)

trace_hydro = go.Scatter(
    x=filtered_data['Year'],
    y=filtered_data['Electricity from hydro - TWh'],
    mode='markers+lines',
    name='Hydro',
    marker=dict(symbol='circle', size=8)
)

trace_other = go.Scatter(
    x=filtered_data['Year'],
    y=filtered_data['Other renewables including bioenergy - TWh'],
    mode='markers+lines',
    name='Other',
    marker=dict(symbol='circle', size=8)
)

# Combine traces into data list
traces = [trace_wind, trace_solar, trace_hydro, trace_other]

# Define layout
layout = go.Layout(
    title='Renewable Energy Production in Lithuania',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Electricity Production (TWh)'),
    hovermode='closest',
    legend=dict(orientation="h")
)

# Reference data source
source_annotation = go.layout.Annotation(
    text='Source: Our World in Data 2024',
    xref='paper', yref='paper',
    x=0.95, y=-0.15,
    showarrow=False,
    font=dict(size=10, color='grey')
)

layout['annotations'] = [source_annotation]

fig14 = go.Figure(data=traces, layout=layout)
fig14.update_layout(template="plotly_dark")
fig14.show()

### Electricity production by source in Lithuania

In [6]:

# Filter data 
filtered_data_lt = df_share_electricity_renewables[df_share_electricity_renewables['Entity'] == 'Lithuania']
filtered_data_lv = df_share_electricity_renewables[df_share_electricity_renewables['Entity'] == 'Latvia']
filtered_data_ee = df_share_electricity_renewables[df_share_electricity_renewables['Entity'] == 'Estonia']
filtered_data_pl = df_share_electricity_renewables[df_share_electricity_renewables['Entity'] == 'Poland']
filtered_data_eu = df_share_electricity_renewables[df_share_electricity_renewables['Entity'] == 'European Union (27)']

# Define traces for each country
trace_lt = go.Scatter(
    x=filtered_data_lt['Year'],
    y=filtered_data_lt['Renewables - % electricity'],
    mode='markers+lines',
    name='Lithuania',
    marker=dict(symbol='circle', size=8)
)

trace_lv = go.Scatter(
    x=filtered_data_lv['Year'],
    y=filtered_data_lv['Renewables - % electricity'],
    mode='markers+lines',
    name='Latvia',
    marker=dict(symbol='circle', size=8)
)

trace_ee = go.Scatter(
    x=filtered_data_ee['Year'],
    y=filtered_data_ee['Renewables - % electricity'],
    mode='markers+lines',
    name='Estonia',
    marker=dict(symbol='circle', size=8)
)

trace_pl = go.Scatter(
    x=filtered_data_pl['Year'],
    y=filtered_data_pl['Renewables - % electricity'],
    mode='markers+lines',
    name='Poland',
    marker=dict(symbol='circle', size=8)
)

trace_eu = go.Scatter(
    x=filtered_data_eu['Year'],
    y=filtered_data_eu['Renewables - % electricity'],
    mode='markers+lines',
    name='European Union (27)',
    marker=dict(symbol='circle', size=8)
)

# Combine traces into data list
traces = [trace_lt, trace_lv, trace_ee, trace_pl, trace_eu]

layout = go.Layout(
    title='Share of electricity production from renewables in the region',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Renewable Energy Share (%)'),
    hovermode='closest',
    legend=dict(orientation="h"),
    height=600
)
fig16 = go.Figure(data=traces, layout=layout)

fig16.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0.95, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)
fig16.update_layout(template="plotly_dark")
fig16.show()

### Share of electricity production from renewables regional comparison

In [7]:
# Filter data for the selected countries
filtered_data = df_share_electricity_renewables[df_share_electricity_renewables['Entity'].isin(['Lithuania', 'Latvia', 'Estonia', 'Poland', 'Finland', 'Germany', 'Denmark', 'Norway', 'Sweden', 'Belarus', 'Ukraine', 'Russia'])]

# Choropleth map
fig17 = px.choropleth(filtered_data, 
    locations='Code', 
    color='Renewables - % electricity',  # Data to be color-coded
    hover_name='Entity',  # Data to be displayed on hover
    title='Share of electricity production from renewables in the region',
    labels={'Renewables - % electricity': 'Renewable Energy Share (%)'}
    )

# Initial center, zoom level
fig17.update_geos(
    center=dict(lon=24, lat=56),  
    projection_rotation=dict(lon=30, lat=10, roll=0), 
    lonaxis_range=[5, 40],
    lataxis_range=[50, 70],  
    scope='europe',  
    showcountries=True 
)

fig17.update_layout(
    height=600,  
    template="plotly_dark"
)

fig17.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)
fig17.show()

In [8]:
# Filter data for specific countries
countries_list = ['Lithuania', 'Latvia', 'Estonia', 'Russia', 'Ukraine', 'Poland', 'Finland']
filtered_data = df_share_electricity_imports[df_share_electricity_imports['Entity'].isin(countries_list)]

fig18 = go.Figure()

# Plot energy imports for each country
for country in countries_list:
    country_data = filtered_data[filtered_data['Entity'] == country]
    fig18.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Net electricity imports as a share of demand - %'], 
                             mode='lines', name=country))

fig18.update_layout(title='Net electricity imports as a share of demand - %',
    xaxis_title='Year',
    yaxis_title='Energy Imports (share of demand - %)',
    showlegend=True, 
    height=600,
    template="plotly_dark")

fig18.add_annotation(
    text="Source: Our World in Data 2024",
    xref="paper", yref="paper",
    x=0, y=-0.15,
    showarrow=False,
    font=dict(size=10, color="grey")
)

fig18.show()

In [9]:
import plotly.express as px
from jinja2 import Template
import json

figs = [fig13, fig14, fig15, fig16, fig17, fig18]  

# Convert each Plotly figure to JSON
fig_jsons = [fig.to_json() for fig in figs]

# Load the Jinja2 template
input_template_path = "html_samples/input.html"
with open(input_template_path) as template_file:
    j2_template = Template(template_file.read())
rendered_html = j2_template.render(fig_jsons=fig_jsons)

# Save HTML
output_html_path = "html_samples/Baltic_region.html"
with open(output_html_path, "w", encoding="utf-8") as output_file:
    output_file.write(rendered_html)
